In [17]:
import pandas as pd
import numpy as np # linear algebra

# Replace 'path_to_dataset' with the actual path to your dataset file
df = pd.read_excel('LastDataset1(neww).xlsx')
df.shape

(2088, 95)

In [18]:
X = df.drop(columns=['JS_Job_sector'])
y = df['JS_Job_sector']

categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int', 'float']).columns

# Create DataFrames for categorical and numerical columns
X_categorical = X[categorical_columns]
X_numerical = X[numerical_columns]

from sklearn.preprocessing import LabelEncoder



# Perform label encoding only on columns with dtype object
label_encoder = LabelEncoder()
X_encoded = X_categorical.copy()
for col in categorical_columns:
    X_encoded[col] = X[col].astype(str)  # Ensure the column is of string dtype before label encoding
    X_encoded[col] = label_encoder.fit_transform(X_encoded[col])

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


# Initialize LabelEncoder for the target variable y
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)
# Merge with original X containing int and float columns
X_final = pd.concat([X_encoded, X.select_dtypes(include=['int', 'float'])], axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_encoded, test_size=0.2, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

smote = SMOTE(random_state=42)
X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled_borderline, y_train_resampled_borderline = borderline_smote.fit_resample(X_train, y_train)


## SMOTE + Bordeline SMOTE

RF

In [35]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the classifier
rf_classifier.fit(X_train_resampled_smote, y_train_resampled_smote)
# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 100}
Accuracy (Best Random Forest with SMOTE): 0.8205741626794258
Classification Report (Best Random Forest with SMOTE):
               precision    recall  f1-score   support

           0      0.721     0.880     0.793        50
           1      0.784     0.667     0.721        60
           2      0.733     0.786     0.759        14
           3      0.000     0.000     0.000         2
           4      0.886     0.921     0.903        76
           5      0.778     0.538     0.636        13
           6      0.792     1.000     0.884        19
           7      0.714     0.625     0.667         8
           8      0.000     0.000     0.000         5
           9      1.000     1.000     1.000        94
          10      0.571     0.286     0.381        14
          11      0.500     0.655     0.567        29
          12      0.968     0.882     0.923        34

    a

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [34]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize Borderline SMOTE

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with Borderline SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with Borderline SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with Borderline SMOTE): 0.8205741626794258
Classification Report (Best Random Forest with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.782     0.860     0.819        50
           1      0.755     0.667     0.708        60
           2      0.647     0.786     0.710        14
           3      0.000     0.000     0.000         2
           4      0.910     0.934     0.922        76
           5      0.889     0.615     0.727        13
           6      0.692     0.947     0.800        19
           7      0.500     0.375     0.429         8
           8      0.000     0.000     0.000         5
           9      1.000     1.000     1.000        94
          10      1.000     0.214     0.353        14
          11      0.468     0.759     0.579        29
          12      1.000     0.882     

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Gaussian NB

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 1.778e-07}
Accuracy (Best GaussianNB with SMOTE): 0.631578947368421
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.634     0.520     0.571        50
           1      0.607     0.567     0.586        60
           2      0.312     0.714     0.435        14
           3      0.000     0.000     0.000         2
           4      1.000     0.618     0.764        76
           5      0.150     0.231     0.182        13
           6      0.481     0.684     0.565        19
           7      1.000     0.125     0.222         8
           8      0.000     0.000     0.000         5
           9      1.000     1.000     1.000        94
          10      0.087     0.143     0.108        14
          11      0.256     0.345     0.294        29
          12      0.706     0.706     0.706        34

    accuracy              

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 1.778e-07}
Accuracy (Best GaussianNB with SMOTE): 0.6578947368421053
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.608     0.620     0.614        50
           1      0.636     0.583     0.609        60
           2      0.310     0.643     0.419        14
           3      0.125     0.500     0.200         2
           4      0.978     0.592     0.738        76
           5      0.412     0.538     0.467        13
           6      0.444     0.632     0.522        19
           7      1.000     0.125     0.222         8
           8      0.000     0.000     0.000         5
           9      1.000     1.000     1.000        94
          10      0.238     0.357     0.286        14
          11      0.312     0.345     0.328        29
          12      0.735     0.735     0.735        34

    accuracy             

DT

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.8444976076555024
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.889     0.960     0.923        50
           1      0.906     0.800     0.850        60
           2      0.875     0.500     0.636        14
           3      0.000     0.000     0.000         2
           4      1.000     0.908     0.952        76
           5      0.571     0.923     0.706        13
           6      0.842     0.842     0.842        19
           7      0.857     0.750     0.800         8
           8      0.400     0.400     0.400         5
           9      1.000     1.000     1.000        94
          10      0.400     0.429     0.414        14
          11      0.429     0.517     0.469        29
          12      0.857     0.882     0.870        34

    accuracy                

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.8588516746411483
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.885     0.920     0.902        50
           1      0.855     0.783     0.817        60
           2      0.471     0.571     0.516        14
           3      0.333     0.500     0.400         2
           4      1.000     0.908     0.952        76
           5      0.647     0.846     0.733        13
           6      0.864     1.000     0.927        19
           7      1.000     0.750     0.857         8
           8      0.250     0.200     0.222         5
           9      1.000     1.000     1.000        94
          10      0.500     0.571     0.533        14
          11      0.643     0.621     0.632        29
          12      0.886     0.912     0.899        34

    accuracy                

KNN


In [39]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE to the training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with SMOTE): 0.5047846889952153
Classification Report (Best KNN with SMOTE):
               precision    recall  f1-score   support

           0      0.400     0.440     0.419        50
           1      0.400     0.233     0.295        60
           2      0.250     0.286     0.267        14
           3      0.000     0.000     0.000         2
           4      0.667     0.368     0.475        76
           5      0.600     0.462     0.522        13
           6      0.289     0.579     0.386        19
           7      0.125     0.250     0.167         8
           8      0.000     0.000     0.000         5
           9      0.979     0.989     0.984        94
          10      0.077     0.071     0.074        14
          11      0.235     0.276     0.254        29
          12      0.611     0.647     0.629        34

    accuracy                          0.505      

In [40]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Borderline SMOTE to the training data only
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with Borderline SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with Borderline SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with Borderline SMOTE): 0.5430622009569378
Classification Report (Best KNN with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.441     0.520     0.477        50
           1      0.500     0.267     0.348        60
           2      0.333     0.357     0.345        14
           3      0.000     0.000     0.000         2
           4      0.643     0.474     0.545        76
           5      0.538     0.538     0.538        13
           6      0.295     0.684     0.413        19
           7      0.200     0.250     0.222         8
           8      0.000     0.000     0.000         5
           9      0.979     0.989     0.984        94
          10      0.000     0.000     0.000        14
          11      0.194     0.207     0.200        29
          12      0.590     0.676     0.630        34

    accuracy               

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVM

In [44]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE



# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with SMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with SMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with SMOTE): {'C': 100, 'gamma': 0.01}
Accuracy (SVM with SMOTE): 0.6985645933014354
Classification Report (SVM with SMOTE):
               precision    recall  f1-score   support

           0      0.661     0.780     0.716        50
           1      0.617     0.483     0.542        60
           2      0.357     0.357     0.357        14
           3      0.000     0.000     0.000         2
           4      0.677     0.882     0.766        76
           5      1.000     0.615     0.762        13
           6      0.500     0.421     0.457        19
           7      0.250     0.125     0.167         8
           8      0.000     0.000     0.000         5
           9      0.989     1.000     0.995        94
          10      0.500     0.143     0.222        14
          11      0.379     0.379     0.379        29
          12      0.718     0.824     0.767        34

    accuracy                      

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [45]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with BorderlineSMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with BorderlineSMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with BorderlineSMOTE): {'C': 10, 'gamma': 0.01}
Accuracy (SVM with BorderlineSMOTE): 0.6985645933014354
Classification Report (SVM with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.709     0.780     0.743        50
           1      0.612     0.500     0.550        60
           2      0.462     0.429     0.444        14
           3      0.000     0.000     0.000         2
           4      0.650     0.882     0.749        76
           5      1.000     0.615     0.762        13
           6      0.467     0.368     0.412        19
           7      0.250     0.125     0.167         8
           8      0.000     0.000     0.000         5
           9      0.989     1.000     0.995        94
          10      0.500     0.143     0.222        14
          11      0.379     0.379     0.379        29
          12      0.692     0.794     0.740        34

    a

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LR

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with SMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with SMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with SMOTE): {'C': 10, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with SMOTE): 0.6172248803827751
Classification Report (Logistic Regression with SMOTE):
               precision    recall  f1-score   support

           0      0.737     0.560     0.636        50
           1      0.583     0.467     0.519        60
           2      0.409     0.643     0.500        14
           3      0.000     0.000     0.000         2
           4      0.758     0.618     0.681        76
           5      0.417     0.385     0.400        13
           6      0.409     0.474     0.439        19
           7      0.095     0.250     0.138         8
           8      0.000     0.000     0.000         5
           9      0.989     1.000     0.995        94
          10      0.250     0.286     0.267        14
          11      0.370     0.345     0.357        29
          12      0.550     0.647     0

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with BorderlineSMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with BorderlineSMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with BorderlineSMOTE): {'C': 1, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with BorderlineSMOTE): 0.6411483253588517
Classification Report (Logistic Regression with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.732     0.600     0.659        50
           1      0.636     0.467     0.538        60
           2      0.444     0.571     0.500        14
           3      0.000     0.000     0.000         2
           4      0.778     0.645     0.705        76
           5      0.556     0.385     0.455        13
           6      0.481     0.684     0.565        19
           7      0.133     0.250     0.174         8
           8      0.000     0.000     0.000         5
           9      1.000     0.989     0.995        94
          10      0.118     0.143     0.129        14
          11      0.414     0.414     0.414        29
          

XGBoost

In [49]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': 4}
Accuracy (XGBoost with SMOTE): 0.9210526315789473
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.959     0.940     0.949        50
           1      0.871     0.900     0.885        60
           2      1.000     0.714     0.833        14
           3      1.000     0.500     0.667         2
           4      1.000     0.987     0.993        76
           5      1.000     0.923     0.960        13
           6      0.900     0.947     0.923        19
           7      0.889     1.000     0.941         8
           8      0.500     0.600     0.545         5
           9      1.000     1.000     1.000        94
          10      0.700     0.500     0.583        14
          11      0.667     0.828     0.738        29
          12      0.941     0.941     0.941        34

    accuracy   

In [50]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': 3}
Accuracy (XGBoost with SMOTE): 0.9330143540669856
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.923     0.960     0.941        50
           1      0.898     0.883     0.891        60
           2      0.846     0.786     0.815        14
           3      0.000     0.000     0.000         2
           4      1.000     1.000     1.000        76
           5      1.000     0.923     0.960        13
           6      0.864     1.000     0.927        19
           7      1.000     1.000     1.000         8
           8      1.000     0.600     0.750         5
           9      1.000     1.000     1.000        94
          10      1.000     0.643     0.783        14
          11      0.714     0.862     0.781        29
          12      0.914     0.941     0.928        34

    accuracy   

In [51]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost classifier with default parameters
xgb_classifier_baseline = XGBClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
xgb_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb_baseline = xgb_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_xgb_baseline = accuracy_score(y_test, y_pred_xgb_baseline)
classification_report_result_xgb_baseline = classification_report(y_test, y_pred_xgb_baseline, digits=3)

# Print results
print("Baseline XGBoost Model:")
print(f"Accuracy: {accuracy_xgb_baseline}")
print("Classification Report:\n", classification_report_result_xgb_baseline)


Baseline XGBoost Model:
Accuracy: 0.9832535885167464
Classification Report:
               precision    recall  f1-score   support

           0      1.000     0.960     0.980        50
           1      0.967     0.983     0.975        60
           2      0.929     0.929     0.929        14
           3      0.000     0.000     0.000         2
           4      1.000     1.000     1.000        76
           5      1.000     0.923     0.960        13
           6      0.950     1.000     0.974        19
           7      1.000     1.000     1.000         8
           8      1.000     1.000     1.000         5
           9      1.000     1.000     1.000        94
          10      0.875     1.000     0.933        14
          11      0.967     1.000     0.983        29
          12      1.000     1.000     1.000        34

    accuracy                          0.983       418
   macro avg      0.899     0.907     0.903       418
weighted avg      0.979     0.983     0.981       418



AdaBoost

In [52]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 50}
Accuracy (AdaBoost with SMOTE): 0.7416267942583732
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.771     0.740     0.755        50
           1      0.727     0.533     0.615        60
           2      0.282     0.786     0.415        14
           3      0.111     1.000     0.200         2
           4      1.000     1.000     1.000        76
           5      1.000     0.846     0.917        13
           6      0.857     0.632     0.727        19
           7      0.000     0.000     0.000         8
           8      0.400     0.400     0.400         5
           9      1.000     0.989     0.995        94
          10      0.074     0.143     0.098        14
          11      0.389     0.241     0.298        29
          12      1.000     0.735     0.847        34

  

In [53]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 50}
Accuracy (AdaBoost with SMOTE): 0.5263157894736842
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.625     0.100     0.172        50
           1      0.256     0.683     0.373        60
           2      0.224     0.786     0.349        14
           3      0.000     0.000     0.000         2
           4      1.000     1.000     1.000        76
           5      1.000     0.846     0.917        13
           6      0.857     0.632     0.727        19
           7      0.000     0.000     0.000         8
           8      0.143     0.400     0.211         5
           9      1.000     0.138     0.243        94
          10      0.286     0.143     0.190        14
          11      0.525     0.724     0.609        29
          12      1.000     0.765     0.867        34

  

In [54]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the AdaBoost classifier with default parameters
ada_classifier_baseline = AdaBoostClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
ada_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ada_baseline = ada_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_ada_baseline = accuracy_score(y_test, y_pred_ada_baseline)
classification_report_result_ada_baseline = classification_report(y_test, y_pred_ada_baseline, digits=3)

# Print results
print("Baseline AdaBoost Model:")
print(f"Accuracy: {accuracy_ada_baseline}")
print("Classification Report:\n", classification_report_result_ada_baseline)


Baseline AdaBoost Model:
Accuracy: 0.4043062200956938
Classification Report:
               precision    recall  f1-score   support

           0      0.000     0.000     0.000        50
           1      0.000     0.000     0.000        60
           2      0.000     0.000     0.000        14
           3      0.000     0.000     0.000         2
           4      0.235     1.000     0.380        76
           5      0.000     0.000     0.000        13
           6      0.000     0.000     0.000        19
           7      0.000     0.000     0.000         8
           8      0.000     0.000     0.000         5
           9      1.000     0.989     0.995        94
          10      0.000     0.000     0.000        14
          11      0.000     0.000     0.000        29
          12      0.000     0.000     0.000        34

    accuracy                          0.404       418
   macro avg      0.095     0.153     0.106       418
weighted avg      0.268     0.404     0.293       418



CatBoost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined

# Initialize the CatBoost classifier with default parameters
catboost_classifier = CatBoostClassifier(random_state=42, verbose=0)

# Define the parameter grid for CatBoost
param_grid = {
    'depth': [2, 3, 4, 5, 6],
    'random_strength': [1, 2, 3, 4, 5],
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=catboost_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1,  # Use all available cores
)

# Fit GridSearchCV on SMOTE resampled data
# This step needs to be executed within tqdm for the progress bar to display properly
with tqdm(total=len(param_grid['depth']) * len(param_grid['random_strength'])) as pbar:
    grid_search_results = grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)
    pbar.update()
    best_catboost_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_catboost = best_catboost_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_catboost = accuracy_score(y_test, y_pred_best_catboost)
classification_report_result_best_catboost = classification_report(y_test, y_pred_best_catboost, digits=3)

# Print results
print(f"Best Parameters (CatBoost with SMOTE): {grid_search_results.best_params_}")
print(f"Accuracy (CatBoost with SMOTE): {accuracy_best_catboost}")
print("Classification Report (CatBoost with SMOTE):\n", classification_report_result_best_catboost)


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import BorderlineSMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Initialize the CatBoost classifier with default parameters
catboost_classifier = CatBoostClassifier(random_state=42, verbose=0)

# Define the parameter grid for CatBoost
param_grid = {
    'depth': [2, 3, 4, 5, 6],
    'random_strength': [1, 2, 3, 4, 5],
}

# Initialize BorderlineSMOTE for oversampling
borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled_borderline, y_train_resampled_borderline = borderline_smote.fit_resample(X_train, y_train)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=catboost_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on BorderlineSMOTE resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_catboost_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_catboost = best_catboost_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_catboost = accuracy_score(y_test, y_pred_best_catboost)
classification_report_result_best_catboost = classification_report(y_test, y_pred_best_catboost, digits=3)

# Print results
print(f"Best Parameters (CatBoost with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (CatBoost with BorderlineSMOTE): {accuracy_best_catboost}")
print("Classification Report (CatBoost with BorderlineSMOTE):\n", classification_report_result_best_catboost)


In [62]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Assuming X_train_resampled_smote, y_train_resampled_smote, X_test, y_test are defined

# Initialize the CatBoost classifier with default parameters
catboost_classifier = CatBoostClassifier(random_state=42)


# Fit the CatBoost classifier
catboost_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_catboost = catboost_classifier.predict(X_test)

# Evaluate the model
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
classification_report_catboost = classification_report(y_test, y_pred_catboost, digits=3)

# Print results
print(f"Accuracy (CatBoost Baseline with SMOTE): {accuracy_catboost}")
print("Classification Report (CatBoost Baseline with SMOTE):\n", classification_report_catboost)


Learning rate set to 0.081224
0:	learn: 2.0773149	total: 21.8ms	remaining: 21.8s
1:	learn: 1.8651145	total: 40.6ms	remaining: 20.2s
2:	learn: 1.6853340	total: 62ms	remaining: 20.6s
3:	learn: 1.5646119	total: 82.5ms	remaining: 20.5s
4:	learn: 1.4674953	total: 100ms	remaining: 20s
5:	learn: 1.3992932	total: 122ms	remaining: 20.3s
6:	learn: 1.3304786	total: 141ms	remaining: 20.1s
7:	learn: 1.2678409	total: 159ms	remaining: 19.7s
8:	learn: 1.2066911	total: 178ms	remaining: 19.6s
9:	learn: 1.1562567	total: 197ms	remaining: 19.5s
10:	learn: 1.1085275	total: 218ms	remaining: 19.6s
11:	learn: 1.0729234	total: 285ms	remaining: 23.5s
12:	learn: 1.0332360	total: 322ms	remaining: 24.4s
13:	learn: 0.9979363	total: 340ms	remaining: 24s
14:	learn: 0.9628940	total: 361ms	remaining: 23.7s
15:	learn: 0.9342974	total: 381ms	remaining: 23.4s
16:	learn: 0.9099918	total: 399ms	remaining: 23.1s
17:	learn: 0.8892918	total: 418ms	remaining: 22.8s
18:	learn: 0.8683293	total: 436ms	remaining: 22.5s
19:	learn: 0.

LGBM

In [71]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007698 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2965
[LightGBM] [Info] Number of data points in the train set: 4706, number of used features: 92
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[L

In [72]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002025 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2946
[LightGBM] [Info] Number of data points in the train set: 4352, number of used features: 92
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -6.298949
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [